In [5]:
import re
import numpy as np
import json

In [3]:
filename = input("Insert file name to read: \n")

with open(filename) as file:
    data = file.readlines()

header = data[0]
content = data[1:]

In [7]:
header_pattern = re.compile(
    r"Número,Nome,Curso(,?Notas?(?P<Notas>{\d,\d})?(?:::)?(?P<agregator>\w*)?(?:.*))?")
content_pattern = re.compile(
    r"(?P<Número>\d*),(?P<Nome>.*[^\W\d_]),(?P<curso>.*[^\W\d_]),?(?P<Notas>[\d,]*)?\n?")

In [8]:
header_match = re.search(header_pattern, header)
limit = (0, 0)
hasAgragator = False


if header_match:
    info_header = header_match.groupdict()
    hasAgragator = True if info_header.get('agregator') else False

    if info_header['Notas'] is not None:
        if ',' not in info_header['Notas']:
            limit = (int((info_header['Notas'])[1]),
                     int((info_header['Notas'])[1]))
        else:
            limit = (int((info_header['Notas'])[1]),
                     int((info_header['Notas'])[-2]))


In [9]:
data_dict = dict()
grades_list = []
store = 0

for line in content:
    content_match = re.fullmatch(content_pattern, line)

    if content_pattern:
        info_content = content_match.groupdict()
        grades_list = list(filter(str.strip, info_content['Notas'].split(',')))
        if limit[0] <= len(grades_list) <= limit[1]:
            if hasAgragator:
                agregator_type = info_header['agregator']
                if agregator_type == 'sum':
                    for num in grades_list:
                        store += int(num)
                    info_content['Notas'] = store
                elif agregator_type == 'media':
                    tmp = []
                    for i, num in enumerate(grades_list):
                        tmp.append(int(num))

                    info_content['Notas'] = np.average(tmp)
            
            data_dict[info_content['Número']] = info_content


In [10]:
try:
    output = data_dict
    with open('alunos.json', 'w') as file:
        json.dump(output, file, indent=4)
    print('JSON file created sucessfully!')
except Exception as e:
    print(e)

JSON file created sucessfully!
